# 데이터 변환

* Last updated 20190909MON1800 20181009_20170421_20161125

## S.1 학습내용

### S.1.1 목표

* Spark ETL을 할 수 있다.
* Spark를 사용하여 구조적 데이터를 분석할 수 있다.
* Spark를 사용하여 텍스트 분석을 할 수 있다.
* Spark를 사용하여 추천을 할 수 있다.
* Spark를 사용하여 그래프 분석을 할 수 있다.
* 시각화
    * matplotlib
    * interactive Bokeh


### S.1.2 목차

* S.2 IPython Notebook에서 SparkSession 생성하기
* S.3 데이터 타잎
* S.3.1 vectors
* S.3.2 labeled point
* S.3.3 maxtrix
* S.3.4 libsvm format
* S.4 통계

* S.5 변환 
* S.5.1 모델의 입력데이터로 변환
* S.5.2 Python을 사용한 단어 빈도 계산
* S.5.3 scikit-learn TF-IDF 
* S.5.4 StringIndexer
* S.5.5 Tokenizer
* S.5.6 RegTokenizer
* S.5.7 Stopwords
* S.5.8 CountVectorizer
* S.5.9 TF-IDF
* S.5.10 Word2Vec
* S.5.11 NGram
* S.5.12 연속데이터의 변환
* 5.5.13 VectorAssembler
* S.5.14 Pipeline

* S.6 머신러닝
* S.6.1 왜 머신러닝?
* S.6.2 라이브러리
* S.6.3 supervised vs unsupervised
* S.6.4 회귀분석
* S.6.5 군집화
* S.6.6 분류
* S.6.7 추천

* S.7 연속 데이터 분석
* S.7.1 데이터
* S.7.2 변환
* S.7.3 KMeans
* S.7.4 Regression
* S.8 구조적 데이터 분석
* S.8.1 데이터
* S.8.2 변환
* S.8.3 LogisticRegression
* S.8.4 svm
* S.8.5 Decision Tree
* S.8.6 Naive Bayesian
* S.9 텍스트 분석
* S.9.1 데이터
* S.9.2 변환
* S.9.3 LogisticRegression
* S.9.4 Decision Tree
* S.9.5 Naive Bayesian
* S.9.6 svm
* S.9.7 LDA
* S.10 추천
* S.11 scikit-learn
* S.12 그래프 분석

### S.1.3 문제 

* 문제 S-1: 훈련데이터 만들기
* 문제 S-2: Kolmogorov-Smirnov 검증
* 문제 S-3: 평균, 표준편차와 같은 기본 통계 값을 구한다.
* 문제 S-4: spark-submit

## S.2 IPython Notebook에서 SparkSession 생성하기


In [2]:
import os
import sys 
os.environ["SPARK_HOME"]=os.path.join(os.environ['HOME'],'Downloads','spark-2.0.0-bin-hadoop2.7')
os.environ["PYLIB"]=os.path.join(os.environ["SPARK_HOME"],'python','lib')
sys.path.insert(0,os.path.join(os.environ["PYLIB"],'py4j-0.10.1-src.zip'))
sys.path.insert(0,os.path.join(os.environ["PYLIB"],'pyspark.zip'))

In [1]:
import pyspark
myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession.builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=myConf)\
    .getOrCreate()

In [2]:
print spark.version

2.2.0


## S.3 Vectors

Spark에서는 **```Vector```**, **```Labeled Point```**, **```Matrix```**를 사용할 수 있다.

구분 | 설명
----------|----------
```Vector``` | ```numpy vector```와 같은 기능을 한다. **dense**와 **sparse** vector로 구분한다.
```Labeled Point``` | 분류를 의미하는 클래스 또는 **label**과 속성 **features** 이 묶인 구조로서, supervised learning에 사용된다.
```Matrix``` | ```numpy matrix```와 같은 특징을 가진다.

이러한 데이터 타잎은 Spark의 **```ml```**, **```mllib```** 패키지 별로 제공되므로, 식별하여 사용한다.
```ml``` 패키지를 사용할 경우에는 아래에서 제공하는 자신의 ```pyspark.ml.linalg.Vector``` 등을 사용해야 한다. ```mllib```도 마찬가지이다.

패키지 | 설명 | 데이터 타잎 예
-------|-------|-----
```mllib``` | RDD API를 제공한다. | ```pyspark.ml.linalg.Vector```, ```pyspark.ml.linalg.Matrix```
```ml``` | DataFrame API를 제공한다. | ```pyspark.mllib.linalg.Vector```, ```pyspark.mllib.linalg.Matrix```


연산
As for now (Spark 1.6.0) pyspark.mllib.linalg.distributed API is limited to basic operations like counting rows/columns and transformations between types.


Neither pyspark.ml.linalg.Matrix nor pyspark.mllib.linalg.Matrix implements matrix multiplication





### S.3.1 vectors

**Vector**는 **dense**와 **sparse**로 구분할 수 있다.
**sparse**는 실제 **값이 없는 요소, '0'을 제거하**여 만든 vector이다.
Spark가 효율적으로 메모리를 사용하기 위해 자동으로 변환하여 사용하기도 한다.
Spark에서 type field (1 바이트 길이)를 통해 식별한다 (0: sparse, 1: dense)

예를 들어, 다음은 dense vector이다.
```python
(160,69,24)
```

이를 sparse vector로 표현하면, 각 컬럼별 해당하는 값을 적는다. 값이 없는 요소가 없으니 더 복잡해 보인다.
```python
(3,[0,1,2],[160.0,69.0,24.0])
```

dense vector | sparse vector
----------|----------
**모든** 행열 값을 가지고 있다. | **인덱스 및 값**의 배열을 별도로 가진다.
빈 값이 별로 없는 경우. | 빈 값이 많은 경우 사용. 
```(160,69,24)``` | ```(3,[0,1,2],[160.0,69.0,24.0])```<br>컬럼 3개, 값이 있는 컬럼, 값
numpy array, Python list를 입력으로 사용 | Vectors.sparse(), SciPy’s csc_matrix

#### Dense vectors

numpy array를 사용해도 dense vector를 만들 수 있다. Spark 내부적으로 **numpy.array**를 사용하고 있다.

In [1]:
import numpy as np

dv = np.array([1.0, 2.1, 3])

Spark에서는 **```RDD mllib```** , **```DataFrame ml```**의 **Vectors**를 사용하여 dense vectors를 만들 수 있다.

In [2]:
from pyspark.mllib.linalg import Vectors

dv1mllib=Vectors.dense([1.0, 2.1, 3])
print dv1mllib, type(dv1mllib)

[1.0,2.1,3.0] <class 'pyspark.mllib.linalg.DenseVector'>


In [3]:
from pyspark.ml.linalg import Vectors

dv1ml=Vectors.dense([1.0, 2.1, 3])
print dv1ml

[1.0,2.1,3.0]


dense vectors는 numpy array와 같은 특징을 가진다.
인덱스로 값을 읽을 수 있다. 또한 반복문에서 사용할 수 있다.

In [4]:
for e in dv1ml:
    print e,

1.0 2.1 3.0


보통 벡터와 같이 **product**, **dot**, **norm**과 같은 벡터 연산을 할 수도 있다.
결과 값은 numpy와 동일하다.

In [5]:
print dv1ml.dot(dv1ml)

14.41


In [6]:
np.dot(dv,dv)

14.41

더하기, 빼기, 곱하기, 나누기 연산은 항목별로 실행한다.

In [7]:
print dv1ml*dv1ml

[1.0,4.41,9.0]


#### Sparse vectors

sparse vectors는 값 중에 0이 포함된 경우 이를 생략한다.
```toArray()``` 함수를 사용하면 sparse에서 dense로 벡터를 변환할 수 있다.

In [23]:
sv1 = Vectors.sparse(3, [1, 2], [1.0, 3.0])
print sv1.toArray()

[ 0.  1.  3.]


많이 사용되는 라이브러리인 scipy.sparse의 **```Compressed Sparse Column```** 형식과 비교해 보자.

행을 보면 0번째에 '1','2' 1번째에 '3', 2번째에 '4','5','6'이므로 **0,0,1,2,2,2**
열을 보면 0번째에 '1', 2번째 '2','3', 0번째 '4', 1번째 '5', 2번째 '6'이므로 **0,2,2,0,1,2**

**행, 열, 데이터를 한 쌍**으로 읽으면 된다.
즉 행 0, 열 0의 위치에 1, 행 0, 열 2의 위치에 2. 이런 식으로 6개의 데이터가 있다.

In [22]:
import numpy as np
import scipy.sparse as sps

row = np.array([0, 0, 1, 2, 2, 2])
col = np.array([0, 2, 2, 0, 1, 2])
data = np.array([1, 2, 3, 4, 5, 6])
mtx = sps.csc_matrix((data, (row, col)), shape=(3, 3))
print mtx.todense()   


[[1 0 2]
 [0 0 3]
 [4 5 6]]


### S.3.2 Labeled Point

#### label, features로 구성

**분류** 및 **회귀분석**에 사용되는 데이터 타잎이다.
**'label'**과 **'features'**로 구성된다.

구분 | 설명
-----|-----
label | supervised learning에서 '구분 값'으로 사용한다. 데이터타잎은 'Double'
features | **sparse**, **dense** 모두 사용할 수 있다.

label 1.0, features [1.0, 2.0, 3.0]으로 LabeledPoint를 만들어 보자.

구분 | 예제 | 설명
-----|-----|-----
label | 1.0 | 구분 값으로 Double 데이터 타잎
features | [1.0, 2.0, 3.0] | **dense vector**

In [29]:
from pyspark.mllib.regression import LabeledPoint

print LabeledPoint(1.0, [1.0, 2.0, 3.0])

(1.0,[1.0,2.0,3.0])


In [8]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vectors

print LabeledPoint(1992, Vectors.sparse(10, {0: 3.0, 1:5.5, 2: 10.0}))

(1992.0,(10,[0,1,2],[3.0,5.5,10.0]))


서로 다른 패키지의 데이터타잎 **```mllib LabeledPoint```**와 **```ml Vectors```**를 혼용하면, 형변환 오류가 발생한다.
이러한 오류는 패키지를 혼용하지 않으면 된다.

```python
Cannot convert type <class 'pyspark.ml.linalg.DenseVector'> into Vector
```

**```dv1mllib```**은 앞서 **```mllib```**로부터 생성된 dense vector이다.

In [31]:
from pyspark.mllib.regression import LabeledPoint

LabeledPoint(1.0, dv1mllib)

LabeledPoint(1.0, [1.0,2.1,3.0])

**```dv1ml```**은 앞서 **```ml```**로부터 생성된 dense vector이다.
```mllib```에서 사용하려면, **```Vectors.fromML()```**를 사용해 ```ml```의 Vectors를 읽어서 ```mllib```로 변환하여 혼용을 막는다.

In [32]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vectors

LabeledPoint(1.0, Vectors.fromML(dv1ml))

LabeledPoint(1.0, [1.0,2.1,3.0])

#### DataFrame에서 Labeled Point

* Python list에서 DataFrame 생성

In [28]:
p = [[1,[1.0,2.0,3.0]],[1,[1.1,2.1,3.1]],[0,[1.2,2.2,3.3]]]
trainDf=spark.createDataFrame(p)
trainDf.collect()

[Row(_1=1, _2=[1.0, 2.0, 3.0]),
 Row(_1=1, _2=[1.1, 2.1, 3.1]),
 Row(_1=0, _2=[1.2, 2.2, 3.3])]

* Python list를 LabeledPoint로 생성하면, 'label'과 'features'의 명칭을 가지도록 생성된다.

In [16]:
from pyspark.mllib.regression import LabeledPoint
p = [LabeledPoint(1,[1.0,2.0,3.0]),
     LabeledPoint(1,[1.1,2.1,3.1]),
     LabeledPoint(0,[1.2,2.2,3.3])]
trainDf=spark.createDataFrame(p)
trainDf.collect()

[Row(features=DenseVector([1.0, 2.0, 3.0]), label=1.0),
 Row(features=DenseVector([1.1, 2.1, 3.1]), label=1.0),
 Row(features=DenseVector([1.2, 2.2, 3.3]), label=0.0)]

mllib.linalg.Vectors를 사용하여 DataFrame을 생성해 보자.

In [10]:
from pyspark.mllib.linalg import Vectors

trainDf = spark.createDataFrame([
    (1.0, Vectors.dense([0.0, 1.1, 0.1])),
    (0.0, Vectors.dense([2.0, 1.0, 1.0])),
    (0.0, Vectors.dense([2.0, 1.3, 1.0])),
    (1.0, Vectors.dense([0.0, 1.2, 0.5]))], ["label", "features"])
trainDf.collect()

[Row(label=1.0, features=DenseVector([0.0, 1.1, 0.1])),
 Row(label=0.0, features=DenseVector([2.0, 1.0, 1.0])),
 Row(label=0.0, features=DenseVector([2.0, 1.3, 1.0])),
 Row(label=1.0, features=DenseVector([0.0, 1.2, 0.5]))]

schema를 사용해서 DataFrame을 생성해 보자.
* 'label'은 **DoubleType**
* 'features'는 **VectorType**

In [17]:
from pyspark.mllib.linalg import SparseVector, VectorUDT
from pyspark.sql.types import StructType, StructField, DoubleType
_rdd = spark.sparkContext.parallelize([
    (0.0, SparseVector(4, {1: 1.0, 3: 5.5})),
    (1.0, SparseVector(4, {0: -1.0, 2: 0.5}))])

schema = StructType([
    StructField("label", DoubleType(), True),
    StructField("features", VectorUDT(), True)
])

In [18]:
trainDf=_rdd.toDF(schema)
trainDf.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



#### sparse에서 dense vector로 변환

방금 생성한 ```trainDf```는 sparse vector이다.
사용자 함수udf User Defined Type를 사용하여 sparse vector를 dense vector로 변환해 보자.
바로 변환할 수 있는 함수 **toDense() 함수를 지원하지 않으므로**, **sparse vector를 ```toArray()``` 함수를 사용해서 dense vector로 변환**한다.

또 ```trainDf```는 mllib RDD에서 변환된 데이터이므로, mllib 라이브러리를 사용한다.

구분 | 설명
-----|-----
from pyspark.mllib.linalg import DenseVector,VectorUDT | mllib
from pyspark.ml.linalg import DenseVector,VectorUDT | ml

다음 명령문을 요소별로 하나씩 설명한다.
```python
udf(lambda x: DenseVector(x.toArray()), VectorUDT())
```
명령문 | 설명
-----|-----
```udf()``` | 사용자정의 함수
```x.toArray()``` | sparse vector로 구성된 trainDf.features를 **```toArray()```**를 사용하여 array로 변환한다.
```VectorUDT()``` | 타잎을 지정한다. 지정하지 않으면 ```StringType```을 기본으로, 자동으로 변환된다.
```DenseVector()``` | array를 dense vector로 변환한다.

In [22]:
from pyspark.sql.functions import udf
#from pyspark.ml.linalg import DenseVector, VectorUDT
from pyspark.mllib.linalg import DenseVector, VectorUDT
#myudf=udf(lambda x: Vectors.dense(x), VectorUDT())
#myudf=udf(lambda x: Vectors.dense(x))
myudf=udf(lambda x: DenseVector(x.toArray()), VectorUDT())
_trainDf2=trainDf.withColumn('dvf',myudf(trainDf.features))

In [23]:
_trainDf2.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- dvf: vector (nullable = true)



In [24]:
_trainDf2.show()

+-----+--------------------+------------------+
|label|            features|               dvf|
+-----+--------------------+------------------+
|  0.0| (4,[1,3],[1.0,5.5])| [0.0,1.0,0.0,5.5]|
|  1.0|(4,[0,2],[-1.0,0.5])|[-1.0,0.0,0.5,0.0]|
+-----+--------------------+------------------+



### S.3.3 maxtrix

* local matrix - pyspark.mllib.linalg.Matrix, Matrices
* distributed matrix
    * pyspark.mllib.linalg.distributed.RowMatrix
    * pyspark.mllib.linalg.distributed.IndexedRow, IndexedRowMatrix
    * pyspark.mllib.linalg.distributed.BlockMatrix

In [21]:
from pyspark.mllib.linalg import Matrix, Matrices

# Create a dense matrix ((1.0, 2.0), (3.0, 4.0), (5.0, 6.0))
dm = Matrices.dense(3, 2, [1, 2, 3, 4, 5, 6])

# Create a sparse matrix ((9.0, 0.0), (0.0, 8.0), (0.0, 6.0))
sm = Matrices.sparse(3, 2, [0, 1, 3], [0, 2, 1], [9, 6, 8])

## 문제 S-1: RDD 훈련데이터 만들기

### 문제

머신러닝은 사람이 경험을 통해 배우는 것과 비슷하게 **과거 데이터로부터 학습**을 한다.
학습이란 어렵게 생각할 필요 없이, 과거 데이터에서 수학적이나 알고리즘을 활용하여 어떤 패턴을 찾아내는 것이다.
spark에서 제공한 **데이터 파일 ```data/mllib/sample_svm_data.txt```을 읽어서 훈련데이터**를 만들어 보자.

```python
1 0 2.52078447201548 0 0 0 2.004684436494304 2.000347299268466 0 2.228387042742021 2.228387042742023 0 0 0 0 0 0
0 2.857738033247042 0 0 2.619965104088255 0 2.004684436494304 2.000347299268466 0 2.228387042742021 2.228387042742023 0 0 0 0 0 0
0 2.857738033247042 0 2.061393766919624 0 0 2.004684436494304 0 0 2.228387042742021 2.228387042742023 0 0 0 0 0 0
1 0 0 2.061393766919624 2.619965104088255 0 2.004684436494304 2.000347299268466 0 0 0 0 2.055002875864414 0 0 0 0
1 2.857738033247042 0 2.061393766919624 2.619965104088255 0 2.004684436494304 0 0 0 0 0 2.055002875864414 0 0 0 0
```

### 해결

데이러틀 읽어 **RDD**를 생성하고, ```label```, ```features```를 구성하여 ```Labeled Point```로 만든다.

### Python으로 파일 읽기

파일로부터 데이터를 읽기 위해, 파일명을 구성하고 ```try except``` 구문으로 입출력 오류를 확인할 수 있다.

In [4]:
import os

try:
    _fp=os.path.join(os.environ["SPARK_HOME"],\
        'data','mllib','sample_svm_data.txt')
except:
    print("An exception occurred")

파일로부터 데이터를 **```readlines()```** 함수로 모두 읽어 온다.
첫 행을 읽으면 label, features로 구성되어 있다.

In [5]:
_f=open(_fp,'r')
_lines=_f.readlines()
_f.close()

In [6]:
print _lines[0]

1 0 2.52078447201548 0 0 0 2.004684436494304 2.000347299268466 0 2.228387042742021 2.228387042742023 0 0 0 0 0 0



### Spark에서 RDD 생성

원본 데이터 ```sample_svm_data.txt```는 공백으로 구분되어 있다.
읽을 대상이 파일이므로, RDD를 사용한다. 각 행을 공백으로 분리하여 읽는다.

In [7]:
_rdd=spark.sparkContext.textFile(_fp)\
    .map(lambda line: [float(x) for x in line.split(' ')])

각 행으로 분리되므로 2차원 리스트가 생성이 된다. 첫째 행을 읽으려면 인덱스를 사용해야 한다.

In [11]:
_rdd.take(2)[0]

[1.0,
 0.0,
 2.52078447201548,
 0.0,
 0.0,
 0.0,
 2.004684436494304,
 2.000347299268466,
 0.0,
 2.228387042742021,
 2.228387042742023,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

### LabeledPoint 생성

위 데이터에서 보듯이 첫 열은 **label**로, 그 나머지는 **features**로 생성한다.

In [12]:
from pyspark.mllib.regression import LabeledPoint

_trainRdd0=_rdd.map(lambda line:LabeledPoint(line[0], line[1:]))

In [13]:
_trainRdd0.take(1)

[LabeledPoint(1.0, [0.0,2.52078447202,0.0,0.0,0.0,2.00468443649,2.00034729927,0.0,2.22838704274,2.22838704274,0.0,0.0,0.0,0.0,0.0,0.0])]

공백을 분리하고, 분리된 데이터를 labeled point로 구성하는 기능을 합쳐서 실행해 본다.

In [14]:
_trainRdd=spark.sparkContext.textFile(_fp)\
    .map(lambda line: [float(x) for x in line.split(' ')])\
    .map(lambda p:LabeledPoint(p[0], p[1:]))

In [15]:
_trainRdd.take(1)

[LabeledPoint(1.0, [0.0,2.52078447202,0.0,0.0,0.0,2.00468443649,2.00034729927,0.0,2.22838704274,2.22838704274,0.0,0.0,0.0,0.0,0.0,0.0])]

### 정리하면

데이터를 변환하는 과정을 함수로 만들었다.
```createLP(line)```는 행 데이터를 받아서 LabeledPoint로 생성하고 있다.

In [29]:
def createLP(line):
    p = [float(x) for x in line.split(' ')]
    return LabeledPoint(p[0], p[1:])

_rdd=spark.sparkContext.textFile(_fp)
trainRdd = _rdd.map(createLP)

trainRdd.take(1)

[LabeledPoint(1.0, [0.0,2.52078447202,0.0,0.0,0.0,2.00468443649,2.00034729927,0.0,2.22838704274,2.22838704274,0.0,0.0,0.0,0.0,0.0,0.0])]

### S.3.4 libsvm format

* svm을 처리하기 위한 데이터 형식이다.
* 0은 label, 나머지는 index:value 쌍으로 구성한다.
```python
[label] [index1]:[value1] [index2]:[value2] ...
[label] [index1]:[value1] [index2]:[value2] ...
```

* 예
```python
0 128:51 129:159 130:253 131:159 132:50 155:48 156:238 157:252 158:252 159:252 160:237 182:54 183:227 184:253 185:252 186:239 187:233 ...
```


In [3]:
fsvm=os.path.join(os.environ["SPARK_HOME"],'data','mllib','sample_libsvm_data.txt')
dfsvm = spark.read.format("libsvm").load(fsvm)

In [4]:
type(dfsvm)

pyspark.sql.dataframe.DataFrame

In [5]:
dfsvm.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



In [6]:
dfsvm.take(1)

[Row(label=0.0, features=SparseVector(692, {127: 51.0, 128: 159.0, 129: 253.0, 130: 159.0, 131: 50.0, 154: 48.0, 155: 238.0, 156: 252.0, 157: 252.0, 158: 252.0, 159: 237.0, 181: 54.0, 182: 227.0, 183: 253.0, 184: 252.0, 185: 239.0, 186: 233.0, 187: 252.0, 188: 57.0, 189: 6.0, 207: 10.0, 208: 60.0, 209: 224.0, 210: 252.0, 211: 253.0, 212: 252.0, 213: 202.0, 214: 84.0, 215: 252.0, 216: 253.0, 217: 122.0, 235: 163.0, 236: 252.0, 237: 252.0, 238: 252.0, 239: 253.0, 240: 252.0, 241: 252.0, 242: 96.0, 243: 189.0, 244: 253.0, 245: 167.0, 262: 51.0, 263: 238.0, 264: 253.0, 265: 253.0, 266: 190.0, 267: 114.0, 268: 253.0, 269: 228.0, 270: 47.0, 271: 79.0, 272: 255.0, 273: 168.0, 289: 48.0, 290: 238.0, 291: 252.0, 292: 252.0, 293: 179.0, 294: 12.0, 295: 75.0, 296: 121.0, 297: 21.0, 300: 253.0, 301: 243.0, 302: 50.0, 316: 38.0, 317: 165.0, 318: 253.0, 319: 233.0, 320: 208.0, 321: 84.0, 328: 253.0, 329: 252.0, 330: 165.0, 343: 7.0, 344: 178.0, 345: 252.0, 346: 240.0, 347: 71.0, 348: 19.0, 349: 28.0

## S.4 통계

* mllib 모듈을 사용한다 'pyspark.mllib.stat'
* 기본 통계
* 가설 검증
* 상관관계 - 키와 몸무게의 상관관계

## 문제 S-2:  Kolmogorov-Smirnov 검증

정규분표 비모수 추정

In [9]:
from pyspark.mllib.stat import Statistics

parallelData = spark.sparkContext.parallelize([1.0, 2.0, 5.0, 4.0, 3.0, 3.3, 5.5])

# run a KS test for the sample versus a standard normal distribution
testResult = Statistics.kolmogorovSmirnovTest(parallelData, "norm", 0, 1)
print(testResult)

Kolmogorov-Smirnov test summary:
degrees of freedom = 0 
statistic = 0.841344746068543 
pValue = 5.06089025353873E-6 
Very strong presumption against null hypothesis: Sample follows theoretical distribution.


## 문제 S-3:  평균, 표준편차와 같은 기본 통계 값을 구한다.

### 문제

균등분포 및 정규분포를 무작위로 생성해 기본통계 값을 계산해 보자.

### 해결

무작위는 발생빈도가 어느 쪽에 치우치지 않는다.
Spark에서 무작위로 균등분포 및 정규분포를 생성하고, 기본통계를 계산한다.

### 분포 생성

DataFrame에서 제공하는 통계 기능을 사용해 본다.
컬럼 3개의 DataFrame을 생성한다.
* 첫 컬럼은 'id', SparkSession.range()를 사용한다.
* 무작위 수를 추출해서, 나머지 컬럼 데이터를 만든다. pyspark.sql.functions 함수를 사용한다.
* rand()는 Uniform분포, randn()은 정규분포를 사용한다.

In [7]:
df = spark.range(0,10)
df.show()
df.select('id')

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+



DataFrame[id: bigint]

통계에 '무작위' 수는 중요하다. 무작위 샘플, 무작위 수를 발생하여 확률에서 빈번하게 사용한다.
**```pyspark.sql.functions```**를 사용하여 생성해 보자.

pyspark.sql.functions | 설명
-----|-----
rand() | 0,1 사이의 균등분포를 생성한다. seed를 넣어서 생성할 수 있다.
randn() | 정규분포를 생성한다. seed를 넣어서 생성할 수 있다.


In [9]:
from pyspark.sql.functions import rand, randn
colUniform = rand(seed=10).alias("uniform")
colNormal=randn(seed=27).alias("normal")
df3=df.select("id", colUniform,colNormal)
df3.show()

+---+-------------------+--------------------+
| id|            uniform|              normal|
+---+-------------------+--------------------+
|  0|0.41371264720975787|  0.5888539012978773|
|  1| 0.7311719281896606|  0.8645537008427937|
|  2| 0.1982919638208397| 0.06157382353970104|
|  3|0.12714181165849525|  0.3623040918178586|
|  4| 0.7604318153406678|-0.49575204523675975|
|  5|0.12030715258495939|  1.0854146699817222|
|  6|0.12131363910425985| -0.5284523629183004|
|  7|0.44292918521277047| -0.4798519469521663|
|  8| 0.8898784253886249| -0.8820294772950535|
|  9|0.03650707717266999| -2.1591956435415334|
+---+-------------------+--------------------+



### 기본 통계

주사위는 이산균등분포의 가장 대표적인 예이다. 각 숫자가 나올 확률은 1/6이다.
정규분포는 평균 0을 중심으로 빈도가 몰려있어 표준편차만큼 퍼진 특징을 가진다.
각 컬럼별로 통계 값을 계산할 수 있다.

In [6]:
df3.describe().show()

+-------+------------------+-------------------+--------------------+
|summary|                id|            uniform|              normal|
+-------+------------------+-------------------+--------------------+
|  count|                10|                 10|                  10|
|   mean|               4.5| 0.5488228646413278|0.009861721586543392|
| stddev|3.0276503540974917| 0.2856822245344392|  1.2126061129356596|
|    min|                 0|0.09430205113458567|  -2.573636861034734|
|    max|                 9| 0.9571919406508957|  1.2524569684217643|
+-------+------------------+-------------------+--------------------+



### freqItems()

a, b, c 세 컬럼을 생성한다.
홀수 행이면 1,2,3으로 짝수 행이면 다른 수열로 DataFrame을 생성해 보자.
이 데이터에 대해 60%이상 발생한 행을 출력해 보자.

In [12]:
df=spark.createDataFrame([(1,2,3) if i%2==0 else (i,2*i,i%4) for i in range(100)],["a","b","c"])
df.show(10)

+---+---+---+
|  a|  b|  c|
+---+---+---+
|  1|  2|  3|
|  1|  2|  1|
|  1|  2|  3|
|  3|  6|  3|
|  1|  2|  3|
|  5| 10|  1|
|  1|  2|  3|
|  7| 14|  3|
|  1|  2|  3|
|  9| 18|  1|
+---+---+---+
only showing top 10 rows



In [13]:
freq = df.stat.freqItems(["a","b","c"],0.6)
print freq.show()

+-----------+-----------+-----------+
|a_freqItems|b_freqItems|c_freqItems|
+-----------+-----------+-----------+
|        [1]|        [2]|        [3]|
+-----------+-----------+-----------+

None


## S.5 변환

Spark는 다양한 입력에서 데이터를 추출, 변환하여 예측, 분류, 군집화, 추천과 같은 모델에 사용하게 된다. 이를 **ETL** (Extract, Transform, Load)이라고 하며 Spark **RDD**는 map-reduce와 같은 **transformation**, **action**을 사용한다.
**DataFrame**도 비슷하게 **Transformer**, **Estimator**를 사용할 수 있다.
**Pipeline**은 여러 Estimator를 묶은 Estimator를 반환한다. 단계적으로 Estimator를 적용하기 위해 사용한다.

변환 기능 | 설명 | 함수
----------|----------|----------
Estimator | 모델의 인자를 설정, 데이터에 적용한다. Transformer를 반환한다.| ```Estimator.fit()```
Transformer | 열을 선택, 변환한다. 그 결과를 DataFrame으로 반환한다. | ```Transformer.transform()```


### S.5.1 모델의 입력데이터로 변환

* 군집화, 회귀분석, 분류, 추천 모델에 사용되는 데이터는 '일련의 수' 또는 '텍스트'로 구성된다.
* 특징을 추출하여 **feature vectors**를 구성한다.
* 분류를 하는 경우에는 **class 또는 label** 값이 필요하다.
* 텍스트는 **'bag of words'**으로 표현한다.
    * 문서는 단어로 구성된다.
    * 단어의 순서는 의미를 가지지 않는다.
    
구분 | 설명 | 예
----------|----------|----------|
corpus | 문서 집합 | "why she had to go", "where she have to go"
document | 레코드 | "why she had to go"
vocabularay | 중복없는 단어 집합 | "why","she","had","to","go","where","have"
word vector | 있다-없다, 단어빈도, TFIDF 사용할 수 있다.<br>dense, sparse 모두 가능하다. | ```[1,1,1,1,1,0,0],[0,1,0,1,1,1,1]```


* 데이터의 구성
    * 데이터 컬럼 명은 'lableCol', 'featureCol'으로 설정해서 사용할 수 있다.
    * 기본 컬럼 명을 사용하는 경우:

구분 | 컬럼 구성
-----|-----
DataFrame | 'label' (DoubleType), 'features' (sparse or dense vectors)
Rdd | LabeledPoint

* 데이터 정련
    * 결측 값, 범위를 벗어나는 outlier, 
    * trainRDD에 마이너스 값 nok?!

### S.5.2 Python을 사용한 단어 빈도 계산

In [1]:
# Let it be lyrics
doc=[
    "When I find myself in times of trouble",
    "Mother Mary comes to me",
    "Speaking words of wisdom, let it be",
    "And in my hour of darkness",
    "She is standing right in front of me",
    "Speaking words of wisdom, let it be",
    "Let it be",
    "Let it be",
    "Let it be",
    "Let it be",
    "Whisper words of wisdom, let it be"
]

문서, 문장, 단어의 계층을 먼저 이해해야 한다.
문서는 문장으로 구성되어 있고, 문장은 단어로 구성되어 있다.
따라서 첫째 반복문은 문서의 각 문장에 대해, 단어로 분리하고 있다.
그 다음 반복문은 각 단어에 대해 빈도를 계산한다.
각 단어가 키가 되는데, **키가 존재하면 빈도를 증가하고, 존재하지 않으면 새로운 키를 생성**한다.

In [2]:
d={}
for sentence in doc:
    words=sentence.split()
    for word in words:
        if word in d:
            d[word]+=1
        else:
            d[word]=1

앞서 단어 빈도는 dictionary d에 저장하였다.
dictionary는 키, 빈도의 쌍으로 저장되어 있어서 ```iteritems()```으로 읽어낼 수 있다.

In [3]:
# Python 2 - 3 compatible code
# for k,v in d.items():
for k,v in d.iteritems():
    print k,v

right 1
be 7
is 1
When 1
it 7
in 3
Mary 1
Speaking 2
standing 1
darkness 1
find 1
wisdom, 3
to 1
Let 4
And 1
I 1
let 3
She 1
words 3
Mother 1
front 1
trouble 1
me 2
myself 1
hour 1
of 6
times 1
Whisper 1
my 1
comes 1


### S.5.3 scikit-learn TF-IDF

* ```TfidfTransformer```는 **TF-IDF(Term Frequency-Inverse Document Frequency)**를 계산한다.
    * 단계 1: Tokenizer를 사용하여 문장을 단어로 분리 
    * 단계 2: CountVectorizer를 사용하여 단어의 빈도수tf를 계산
    * 단계 3: HashingTF를 사용하여 'word vector'를 계산.
    HashingTF은 hash함수에 따라 단어의 고유번호를 생성,
    hash고유번호의 충돌 가능성을 줄이기 위해, 단어 수를 제한할 수 있다.
    * 단계 4: IDF를 계산
    * 단계 5: TF-IDF를 계산 

#### S.5.2.1 TF-IDF 계산

'Let it be'가사 세 번째 줄 **'wisdom' 단어**의 TF-IDF를 계산해보자.
```TfidfVectorizer```를 사용해서 계산하면 그 결과를 아래와 같이 볼 수 있다.
```python
(2,12) 2.09861228867
```

결과에서
**'2'**는 3번째 문서번호, **'12'**는 'wisdom' 단어번호
TF-IDF는 ```2.09861228867```이다.


3번째 문장 "Speaking words of wisdom, let it be"의 word vector를 구성해 본다.
id 값은 모든 문장에서 단어를 추출하고 나서야 부여될 수 있다.

단어 (3행 "Speaking words of wisdom, let it be") | id | 빈도 | 
-----|-----|-----
Speaking | 7 | 1
words | 13 | 1
of | stopword | 0
wisdom | 12 | 1
let | 3 | 1
it | stopword | 0
be | stopword | 0

위 **word vector**를 표로 나타내면 아래와 같다.
행은 문장, 열은 id이다.
**3행은 doc2**이다. 해당하는 **단어 id의 빈도**를 적었다. 다른 행과 열은 이해를 돕기 위해 비워 놓았다.

```doc``` \ 단어 id  | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 |10 |11 |12 |13 |...
------|---|---|---|---|---|---|---|---|---|---|---|---|---|---
```doc 0``` |   |   |   |   |   |   |   |   |   |   |   |   |   |...
```doc 1``` |   |   |   |   |   |   |   |   |   |   |   |   |   |...
```doc 2``` |   |   | 1 |   |   |   | 1 |   |   |   |   | 1 | 1 |...
...   |   |   |   |   |   |   |   |   |   |   |   |   |   |...

* TF-IDF 계산

항목 | 설명 | 예제
-----|-----|-----
tf | term frequency 단어의 빈도 수 | $f_{t,d}$ / (number of words in d) = 1/4 = 0.25<br>(3번째 문서에 stopwords를 제외하면 4개의 단어, wisdom은 1회 나타난다.)
df | document frequency 단어가 나타난 문서 수 | 3 (wisdom이 포함된 문서는 3)
N | number of documents 전체 문서의 수 | 11 (전체의 문서는 11개)
idf | inverse document frequency 단어가 나타난 문서의 비율을 거꾸로 | ln(N+1 / df+1) + 1 = log(12/4) + 1 = 1.09861 + 1<br>0으로 나뉘는 것을 방지하기 위해 **smoothing**, 즉 1을 더한다. 

In [15]:
import math
tf=1./4
df=3.
N=11.
idf=math.log((N+1)/(df+1))+1
print idf

2.09861228867


#### S.5.2.2 sklearn을 사용한 TF-IDF

Spark는 'sklearn'의 TF-IDF와 동일한 방식으로 계산한다.
**```CountVectorizer```**를 사용하여, 문서 x 단어를 표로 계산결과를 출력할 수 있다.
그 다음으로, TF-IDF를 계산할 수 있다. 이 때 (문서id, 단어id) 별로 결과가 출력된다.

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 
vectorizer = CountVectorizer()
print vectorizer.fit_transform(doc).todense()
print vectorizer.vocabulary_

[[0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 1 1 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 1]
 [1 0 0 1 0 0 1 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 1 1 0 0 0 1 0 0 0 1 1 1 0 1 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 1]
 [0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 1]]
{u'and': 0, u'be': 1, u'right': 17, u'whisper': 25, u'is': 8, u'it': 9, u'wisdom': 26, u'me': 12, u'let': 10, u'words': 27, u'in': 7, u'front': 5, u'trouble': 23, u'find': 4, u'standing': 20, u'comes': 2, u'myself': 15, u'darkness': 3, u'hour': 6, u'of': 16, u'when': 24, u'times': 21, u'to': 22, u'she': 18, u'mother': 13, u'my': 14, u'mary': 11, u'

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_df=1.0, min_df=1, stop_words='english',norm = None)

print vectorizer.fit_transform(doc)
print vectorizer.vocabulary_
print vectorizer.idf_


  (0, 10)	2.79175946923
  (0, 9)	2.79175946923
  (1, 0)	2.79175946923
  (1, 4)	2.79175946923
  (1, 5)	2.79175946923
  (2, 3)	1.40546510811
  (2, 12)	2.09861228867
  (2, 13)	2.09861228867
  (2, 7)	2.38629436112
  (3, 1)	2.79175946923
  (3, 2)	2.79175946923
  (4, 6)	2.79175946923
  (4, 8)	2.79175946923
  (5, 3)	1.40546510811
  (5, 12)	2.09861228867
  (5, 13)	2.09861228867
  (5, 7)	2.38629436112
  (6, 3)	1.40546510811
  (7, 3)	1.40546510811
  (8, 3)	1.40546510811
  (9, 3)	1.40546510811
  (10, 11)	2.79175946923
  (10, 3)	1.40546510811
  (10, 12)	2.09861228867
  (10, 13)	2.09861228867
{u'standing': 8, u'right': 6, u'darkness': 1, u'hour': 2, u'whisper': 11, u'times': 9, u'let': 3, u'speaking': 7, u'words': 13, u'mother': 5, u'trouble': 10, u'wisdom': 12, u'mary': 4, u'comes': 0}
[ 2.79175947  2.79175947  2.79175947  1.40546511  2.79175947  2.79175947
  2.79175947  2.38629436  2.79175947  2.79175947  2.79175947  2.79175947
  2.09861229  2.09861229]


* doc를 list of list로 만들어, DataFrame을 생성한다.
* schema는 만들어 주지 않아도 된다.


In [9]:
doc=[
    ["When I find myself in times of trouble"],
    ["Mother Mary comes to me"],
    ["Speaking words of wisdom, let it be"],
    ["And in my hour of darkness"],
    ["She is standing right in front of me"],
    ["Speaking words of wisdom, let it be"],
    [u"우리 Let it be"],
    [u"나 Let it be"],
    [u"너 Let it be"],
    ["Let it be"],
    ["Whisper words of wisdom, let it be"]
]

myDf=spark.createDataFrame(doc,['sent'])
myDf.show(truncate=False)

+--------------------------------------+
|sent                                  |
+--------------------------------------+
|When I find myself in times of trouble|
|Mother Mary comes to me               |
|Speaking words of wisdom, let it be   |
|And in my hour of darkness            |
|She is standing right in front of me  |
|Speaking words of wisdom, let it be   |
|우리 Let it be                          |
|나 Let it be                           |
|너 Let it be                           |
|Let it be                             |
|Whisper words of wisdom, let it be    |
+--------------------------------------+



### S.5.4 StringIndexer

* 문자를 인덱스 값, double로 변환된다.

구분 | 설명 | 예
-----|-----|-----
nominal | 명목 또는 구분 값 cateogry  | 사자, 호랑이, 사람
ordinal | 명목값과 다른 점은 순서가 있다. | 키 low, med, high
interval | 일정한 간격이 있다. | 150-165, 165-180, 180-195


In [10]:
from pyspark.ml.feature import StringIndexer
labelIndexer = StringIndexer(inputCol="sent", outputCol="sentLabel")
model=labelIndexer.fit(myDf)
siDf=model.transform(myDf)
siDf.show()

+--------------------+---------+
|                sent|sentLabel|
+--------------------+---------+
|When I find mysel...|      9.0|
|Mother Mary comes...|      8.0|
|Speaking words of...|      0.0|
|And in my hour of...|      5.0|
|She is standing r...|      4.0|
|Speaking words of...|      0.0|
|        우리 Let it be|      6.0|
|         나 Let it be|      1.0|
|         너 Let it be|      2.0|
|           Let it be|      7.0|
|Whisper words of ...|      3.0|
+--------------------+---------+



#### S.5.5 Tokenizer

* 문장을 단어와 같은 token으로 분리한다.
* 단어는 배열로 구성한다. 요소는 string이다.

In [11]:
from pyspark.ml.feature import Tokenizer
tokenizer = Tokenizer(inputCol="sent", outputCol="words")
tokDf = tokenizer.transform(myDf)
for r in tokDf.select("sent", "words").take(3):
    print r

Row(sent=u'When I find myself in times of trouble', words=[u'when', u'i', u'find', u'myself', u'in', u'times', u'of', u'trouble'])
Row(sent=u'Mother Mary comes to me', words=[u'mother', u'mary', u'comes', u'to', u'me'])
Row(sent=u'Speaking words of wisdom, let it be', words=[u'speaking', u'words', u'of', u'wisdom,', u'let', u'it', u'be'])


#### S.5.6 RegTokenizer

* 단어를 분리하기 위한 패턴을 적용할 수 있다.
* 한글에는 \w 패턴이 적용되지 않는다. 
* whitespace \s 패턴을 적용한다. 공백, TAB, CR, New Line 등이 해당된다.


In [12]:
from pyspark.ml.feature import RegexTokenizer
re = RegexTokenizer(inputCol="sent", outputCol="wordsReg", pattern="\\s+")
reDf=re.transform(myDf)
reDf.show()

+--------------------+--------------------+
|                sent|            wordsReg|
+--------------------+--------------------+
|When I find mysel...|[when, i, find, m...|
|Mother Mary comes...|[mother, mary, co...|
|Speaking words of...|[speaking, words,...|
|And in my hour of...|[and, in, my, hou...|
|She is standing r...|[she, is, standin...|
|Speaking words of...|[speaking, words,...|
|        우리 Let it be|   [우리, let, it, be]|
|         나 Let it be|    [나, let, it, be]|
|         너 Let it be|    [너, let, it, be]|
|           Let it be|       [let, it, be]|
|Whisper words of ...|[whisper, words, ...|
+--------------------+--------------------+



#### S.5.7 Stopwords

* 한 단어 등 불용어.
* http://ir.dcs.gla.ac.uk/resources/linguistic_utils/stop_words

In [13]:
from pyspark.ml.feature import StopWordsRemover
stop = StopWordsRemover(inputCol="wordsReg", outputCol="nostops")

* 현재 stop words에 자신의 것을 추가해서, 재설정한다.

In [14]:
stopwords=list()
_stopwords=stop.getStopWords()
for e in _stopwords:
    stopwords.append(e)

_mystopwords=[u"나",u"너", u"우리"]
for e in _mystopwords:
    stopwords.append(e)
stop.setStopWords(stopwords)

StopWordsRemover_44dba16d6fe647309715

In [15]:
for e in stop.getStopWords():
    print e,

i me my myself we our ours ourselves you your yours yourself yourselves he him his himself she her hers herself it its itself they them their theirs themselves what which who whom this that these those am is are was were be been being have has had having do does did doing a an the and but if or because as until while of at by for with about against between into through during before after above below to from up down in out on off over under again further then once here there when where why how all any both each few more most other some such no nor not only own same so than too very s t can will just don should now d ll m o re ve y ain aren couldn didn doesn hadn hasn haven isn ma mightn mustn needn shan shouldn wasn weren won wouldn 나 너 우리


* 한글의 stop words '너','우리'가 제거되었다.

In [16]:
stopDf=stop.transform(reDf)
stopDf.show()

+--------------------+--------------------+--------------------+
|                sent|            wordsReg|             nostops|
+--------------------+--------------------+--------------------+
|When I find mysel...|[when, i, find, m...|[find, times, tro...|
|Mother Mary comes...|[mother, mary, co...|[mother, mary, co...|
|Speaking words of...|[speaking, words,...|[speaking, words,...|
|And in my hour of...|[and, in, my, hou...|    [hour, darkness]|
|She is standing r...|[she, is, standin...|[standing, right,...|
|Speaking words of...|[speaking, words,...|[speaking, words,...|
|        우리 Let it be|   [우리, let, it, be]|               [let]|
|         나 Let it be|    [나, let, it, be]|               [let]|
|         너 Let it be|    [너, let, it, be]|               [let]|
|           Let it be|       [let, it, be]|               [let]|
|Whisper words of ...|[whisper, words, ...|[whisper, words, ...|
+--------------------+--------------------+--------------------+



#### S.5.8 CountVectorizer

* 입력: a collection of text documents
* 출력: word vector (sparse) vocabulary x TF
* tokenize하고 나서 사용
* minDF
    * 소수점은 비율, 사용된 문서 수/전체 문서 수
        * 정수는 사용된 문서 수, 단어가 몇 개의 문서에 사용되어야 하는지


In [17]:
from pyspark.ml.feature import CountVectorizer
cv = CountVectorizer(inputCol="nostops", outputCol="cv",
    vocabSize=30,minDF=1.0)
cvModel = cv.fit(stopDf)
cvDf = cvModel.transform(stopDf)

cvDf.collect()
cvDf.select('sent','nostops','cv').show()
for v in cvModel.vocabulary:
    print v,

+--------------------+--------------------+--------------------+
|                sent|             nostops|                  cv|
+--------------------+--------------------+--------------------+
|When I find mysel...|[find, times, tro...|(16,[4,9,11],[1.0...|
|Mother Mary comes...|[mother, mary, co...|(16,[7,8,10],[1.0...|
|Speaking words of...|[speaking, words,...|(16,[0,1,2,3],[1....|
|And in my hour of...|    [hour, darkness]|(16,[5,15],[1.0,1...|
|She is standing r...|[standing, right,...|(16,[6,13,14],[1....|
|Speaking words of...|[speaking, words,...|(16,[0,1,2,3],[1....|
|        우리 Let it be|               [let]|      (16,[0],[1.0])|
|         나 Let it be|               [let]|      (16,[0],[1.0])|
|         너 Let it be|               [let]|      (16,[0],[1.0])|
|           Let it be|               [let]|      (16,[0],[1.0])|
|Whisper words of ...|[whisper, words, ...|(16,[0,1,2,12],[1...|
+--------------------+--------------------+--------------------+

let words wisdom, speaki

#### S.5.9 TF-IDF

* Term frequency-inverse document frequency (TF-IDF)
* tokenizer하고 나서 사용해야 함. 
*  HashingTF  고정길이 word vectors.
* IDF 

In [30]:
from pyspark.ml.feature import HashingTF, IDF

hashTF = HashingTF(inputCol="nostops", outputCol="hash", numFeatures=50)
hashDf = hashTF.transform(stopDf)
idf = IDF(inputCol="hash", outputCol="idf")
idfModel = idf.fit(hashDf)
idfDf = idfModel.transform(hashDf)
for e in idfDf.select("nostops","hash").take(10):
    print(e)

Row(nostops=[u'find', u'times', u'trouble'], hash=SparseVector(50, {10: 1.0, 24: 1.0, 43: 1.0}))
Row(nostops=[u'mother', u'mary', u'comes'], hash=SparseVector(50, {1: 1.0, 21: 1.0, 24: 1.0}))
Row(nostops=[u'speaking', u'words', u'wisdom,', u'let'], hash=SparseVector(50, {9: 1.0, 12: 1.0, 14: 1.0, 41: 1.0}))
Row(nostops=[u'hour', u'darkness'], hash=SparseVector(50, {23: 1.0, 27: 1.0}))
Row(nostops=[u'standing', u'right', u'front'], hash=SparseVector(50, {24: 1.0, 43: 1.0, 46: 1.0}))
Row(nostops=[u'speaking', u'words', u'wisdom,', u'let'], hash=SparseVector(50, {9: 1.0, 12: 1.0, 14: 1.0, 41: 1.0}))
Row(nostops=[u'let'], hash=SparseVector(50, {14: 1.0}))
Row(nostops=[u'let'], hash=SparseVector(50, {14: 1.0}))
Row(nostops=[u'let'], hash=SparseVector(50, {14: 1.0}))
Row(nostops=[u'let'], hash=SparseVector(50, {14: 1.0}))


#### S.5.10 Word2Vec

* see wikipedia https://en.wikipedia.org/wiki/Word2vec

In [207]:
from pyspark.ml.feature import Word2Vec
word2Vec = Word2Vec(vectorSize=3,minCount=0,inputCol="words",outputCol="w2v")
model = word2Vec.fit(tokDf)
w2vDf = model.transform(tokDf)
for e in w2vDf.select("w2v").take(3):
    print(e)

Row(w2v=DenseVector([-0.0367, 0.0097, 0.0479]))
Row(w2v=DenseVector([-0.0482, 0.0223, 0.0095]))
Row(w2v=DenseVector([0.052, -0.001, -0.0019]))


#### S.5.11 NGram

* unigram은 한 단어로, bigram은 두 단어로 구성한다.

In [41]:
from pyspark.ml.feature import NGram
ngram = NGram(n=2, inputCol="words", outputCol="ngrams")
ngramDf = ngram.transform(tokDf)
ngramDf.show()
for e in ngramDf.select("words","ngrams").take(3):
    print e

+--------------------+--------------------+--------------------+
|                sent|               words|              ngrams|
+--------------------+--------------------+--------------------+
|When I find mysel...|[when, i, find, m...|[when i, i find, ...|
|Mother Mary comes...|[mother, mary, co...|[mother mary, mar...|
|Speaking words of...|[speaking, words,...|[speaking words, ...|
|And in my hour of...|[and, in, my, hou...|[and in, in my, m...|
|She is standing r...|[she, is, standin...|[she is, is stand...|
|Speaking words of...|[speaking, words,...|[speaking words, ...|
|        우리 Let it be|   [우리, let, it, be]|[우리 let, let it, ...|
|         나 Let it be|    [나, let, it, be]|[나 let, let it, i...|
|         너 Let it be|    [너, let, it, be]|[너 let, let it, i...|
|           Let it be|       [let, it, be]|     [let it, it be]|
|Whisper words of ...|[whisper, words, ...|[whisper words, w...|
+--------------------+--------------------+--------------------+

Row(words=[u'when', u'i'

#### S.5.12 연속데이터의 변환

몸무게(inches), 키(pounds) 데이터를 분석해보자.
이 데이터는 정량, 연속 데이터이다. 
출처는 https://people.sc.fsu.edu/~jburkardt/data/csv/hw_200.csv

```python
1	65.78	112.99
2	71.52	136.49
3	69.40	153.03
4	68.22	142.34
5	67.79	144.30
6	68.70	123.30
7	69.80	141.49
8	70.01	136.46
9	67.90	112.37
10	66.78	120.67
11	66.49	127.45
12	67.62	114.14
13	68.30	125.61
14	67.12	122.46
15	68.28	116.09
16	71.09	140.00
17	66.46	129.50
18	68.65	142.97
19	71.23	137.90
20	67.13	124.04
21	67.83	141.28
22	68.88	143.54
23	63.48	97.90
24	68.42	129.50
25	67.63	141.85
26	67.21	129.72
27	70.84	142.42
28	67.49	131.55
29	66.53	108.33
30	65.44	113.89
31	69.52	103.30
32	65.81	120.75
33	67.82	125.79
34	70.60	136.22
35	71.80	140.10
36	69.21	128.75
37	66.80	141.80
38	67.66	121.23
39	67.81	131.35
40	64.05	106.71
41	68.57	124.36
42	65.18	124.86
43	69.66	139.67
44	67.97	137.37
45	65.98	106.45
46	68.67	128.76
47	66.88	145.68
48	67.70	116.82
49	69.82	143.62
50	69.09	134.93
```

In [67]:
from pyspark.sql.types import *
rdd=spark.sparkContext\
    .textFile(os.path.join('data','ds_spark_heightweight.txt'))

myRdd=rdd.map(lambda line:[float(x) for x in line.split('\t')])
myDf=spark.createDataFrame(myRdd,["id","weight","height"])

In [54]:
myDf.printSchema()

root
 |-- id: double (nullable = true)
 |-- weight: double (nullable = true)
 |-- height: double (nullable = true)



In [68]:
from pyspark.ml.feature import Binarizer
binarizer = Binarizer(threshold=68.0, inputCol="weight", outputCol="weight2")
binDf = binarizer.transform(myDf)
binDf.show(10)

+----+------+------+-------+
|  id|weight|height|weight2|
+----+------+------+-------+
| 1.0| 65.78|112.99|    0.0|
| 2.0| 71.52|136.49|    1.0|
| 3.0|  69.4|153.03|    1.0|
| 4.0| 68.22|142.34|    1.0|
| 5.0| 67.79| 144.3|    0.0|
| 6.0|  68.7| 123.3|    1.0|
| 7.0|  69.8|141.49|    1.0|
| 8.0| 70.01|136.46|    1.0|
| 9.0|  67.9|112.37|    0.0|
|10.0| 66.78|120.67|    0.0|
+----+------+------+-------+
only showing top 10 rows



In [69]:
from pyspark.ml.feature import QuantileDiscretizer

discretizer = QuantileDiscretizer(numBuckets=3, inputCol="height", outputCol="height3")
qdDf = discretizer.fit(binDf).transform(binDf)
qdDf.show(10)


+----+------+------+-------+-------+
|  id|weight|height|weight2|height3|
+----+------+------+-------+-------+
| 1.0| 65.78|112.99|    0.0|    0.0|
| 2.0| 71.52|136.49|    1.0|    1.0|
| 3.0|  69.4|153.03|    1.0|    2.0|
| 4.0| 68.22|142.34|    1.0|    2.0|
| 5.0| 67.79| 144.3|    0.0|    2.0|
| 6.0|  68.7| 123.3|    1.0|    0.0|
| 7.0|  69.8|141.49|    1.0|    2.0|
| 8.0| 70.01|136.46|    1.0|    1.0|
| 9.0|  67.9|112.37|    0.0|    0.0|
|10.0| 66.78|120.67|    0.0|    0.0|
+----+------+------+-------+-------+
only showing top 10 rows



#### 5.5.13 VectorAssembler

* 열을 묶어서 Vector열로 만든다.
* string은 묶을 수 없다.
* pyspark.ml.linalg.Vectors를 사용한다. (주의: pyspark.mllib.linalg.Vectors를 사용하지 않는다.)

In [70]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

va = VectorAssembler(inputCols=["weight2","height3"],outputCol="features")
vaDf = va.transform(qdDf)
vaDf.printSchema()
vaDf.show(5)

root
 |-- id: double (nullable = true)
 |-- weight: double (nullable = true)
 |-- height: double (nullable = true)
 |-- weight2: double (nullable = true)
 |-- height3: double (nullable = true)
 |-- features: vector (nullable = true)

+---+------+------+-------+-------+---------+
| id|weight|height|weight2|height3| features|
+---+------+------+-------+-------+---------+
|1.0| 65.78|112.99|    0.0|    0.0|(2,[],[])|
|2.0| 71.52|136.49|    1.0|    1.0|[1.0,1.0]|
|3.0|  69.4|153.03|    1.0|    2.0|[1.0,2.0]|
|4.0| 68.22|142.34|    1.0|    2.0|[1.0,2.0]|
|5.0| 67.79| 144.3|    0.0|    2.0|[0.0,2.0]|
+---+------+------+-------+-------+---------+
only showing top 5 rows



#### S.5.14 Pipeline

* Pipeline은 여러 작업을 묶어, 순서대로 단계적으로 처리한다.

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.classification import LogisticRegression

df = spark.createDataFrame([
    (0L, "a b c d e spark", 1.0),
    (1L, "b d", 0.0),
    (2L, "spark f g h", 1.0),
    (3L, "hadoop mapreduce", 0.0),
    (4L, "my dog has flea problems. help please.",0.0)
    ], ["id", "text", "label"])

tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
lr = LogisticRegression(maxIter=10, regParam=0.01)
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

# Fit the pipeline to training documents.
model = pipeline.fit(df)
myDf = model.transform(df)